In [5]:
import sys
!{sys.executable} -m pip install psycopg2-binary

import psycopg2 as pg

In [6]:
conn = pg.connect(dbname="sequra", user="docker", password="docker", host="postgresql_sequra", port=5432)

## Load Mock Data into the database

In [7]:
with open("sql-2-fixtures.sql", "r") as file:
    sql = file.read()

cursor = conn.cursor()

try:
    cursor.execute(sql)
    conn.commit()
except (Exception, pg.DatabaseError) as error:
    print("Error: %s" % error)
    conn.rollback()
    cursor.close()

cursor.close()

## Solution

In [8]:
sql = """

WITH stats AS (

  SELECT
    DATE_PART('y', live_on) AS s_year,
    count(m.merchant_id) AS s_year_number  
  FROM
    sql_2.dim_merchants m
  WHERE 
    api_status = 'allowed' AND 
    live_on IS NOT NULL
  GROUP BY 1
  ORDER BY 1 ASC

),
stats_evolution AS (

  SELECT 
    s.s_year AS current_year,
    s.s_year_number AS current_year_number,
    LAG(s.s_year) OVER (ORDER by s.s_year) AS past_year, 
    LAG(s.s_year_number) OVER (ORDER by s.s_year) AS past_year_number 
  FROM
    stats s

)

  SELECT
    se.current_year,
    se.current_year_number,
    se.past_year_number,
    CEIL(((se.current_year_number - se.past_year_number)::DECIMAL/se.past_year_number::DECIMAL)*100)::integer AS growth   
  FROM
    stats_evolution se
  WHERE
    se.past_year IS NOT NULL

"""

cursor = conn.cursor('sql_2_cursor')

try:
    
    cursor.itersize = 5000
    cursor.execute(sql)

    while True:
        rows = cursor.fetchmany(size=5000)

        for row in rows:
            print(row)
        
        if not rows:
            break

except (Exception, pg.DatabaseError) as error:
    print("Error: %s" % error)
    cursor.close()

cursor.close()

(2019.0, 1, 2, -50)
(2020.0, 1, 1, 0)
(2021.0, 1, 1, 0)


In [9]:
conn.close()